# Disney over the years: An analysis of Walt Disney's revenue

#### **Programming in Python for Data Science Final Project**  
Young Ji Tuen

# Introduction

## Question of Interest

In this analysis, I will be investigating a question associated with data regarding the revenue of the Walt Disney Company. I am interested in finding out how the proportion of revenue across different divisions of the company has changed over the years.

This is fascinating as Walt Disney has clearly banched out from its animation roots and has embraced the changing times. As we are living in an increasingly media-saturated world, it might be safe to assume that this has been a smart move on Disney's part. 

Thus, I would expect that while the Walt Disney Company's **studio entertainment division** would contribute a **decreasing** proportion, **all other divisions** would contribute an **increasing** proportion of its yearly revenue. 

## Dataset Description

The Disney Character Success dataset was obtained from [data.world](https://data.world/kgarrett/disney-character-success-00-16) and describes the Walt Disney Company's "characters, box office success & annual gross income". 


The Disney Character Success dataset is composed of **5** tables: `disney-voice-actors.csv`, `disney-director.csv`, `disney_characters.csv`, `disney_revenue_1991-2016.csv`, and `disney_movies_total_gross.csv`. Each table is stored in a `.csv` file and contains different information about Walt Disney's voice actors, directors, movie characters, annual company revenue and movie box office. In this notebook, I will be using the `disney_revenue_1991-2016` table formally described below:

**disney_revenue_1991-2016.csv**

This file contains information on the Walt Disney Company's annual revenue in **millions USD**. It breaks down the annual revenue for each of the company's divisions, which includes its studio entertainment, consumer products, interactive media, parks and resorts, and media networks divisions. 

# Methods and Results

## Data Cleaning

Since I am interested in analyzing the yearly revenue of the Walt Disney Company, I will need to use the `disney_revenue_1991-2016.csv` table.

Let us first import the table.

In [1]:
# import all the required libraries needed for this analysis
import altair as alt
import pandas as pd

# import the required file needed for this analysis
revenue = pd.read_csv("disney_revenue_1991-2016.csv")

Let's take a peek at the table in its original form. 

In [2]:
revenue.head()

,Year,Studio Entertainment[NI 1],Disney Consumer Products[NI 2],Disney Interactive[NI 3][Rev 1],Walt Disney Parks and Resorts,Disney Media Networks,Total
0,1991,2593.0,724.0,NaN,2794.0,NaN,6111
1,1992,3115.0,1081.0,NaN,3306.0,NaN,7502
2,1993,3673.4,1415.1,NaN,3440.7,NaN,8529
3,1994,4793.0,1798.2,NaN,3463.6,359,10414
4,1995,6001.5,2150.0,NaN,3959.8,414,12525


I think that the column headers could first do with some tidying up. We want them to be concise and easy to work with.

In [3]:
#rename columns 
revenue = revenue.rename(columns = {'Year' : 'year', 
                                    'Studio Entertainment[NI 1]' : 'studio_entertainment', 
                                    'Disney Consumer Products[NI 2]' : 'consumer_products', 
                                    'Disney Interactive[NI 3][Rev 1]' : 'interactive_media',
                                    'Walt Disney Parks and Resorts' : 'parks_resorts', 
                                    'Disney Media Networks' : 'media_networks', 
                                    'Total' : 'total'})
revenue.head()

,year,studio_entertainment,consumer_products,interactive_media,parks_resorts,media_networks,total
0,1991,2593.0,724.0,NaN,2794.0,NaN,6111
1,1992,3115.0,1081.0,NaN,3306.0,NaN,7502
2,1993,3673.4,1415.1,NaN,3440.7,NaN,8529
3,1994,4793.0,1798.2,NaN,3463.6,359,10414
4,1995,6001.5,2150.0,NaN,3959.8,414,12525


Much better! 

Let's get some information about the **revenue** table.

In [4]:
revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  26 non-null     int64  
 1   studio_entertainment  25 non-null     float64
 2   consumer_products     24 non-null     float64
 3   interactive_media     12 non-null     float64
 4   parks_resorts         26 non-null     float64
 5   media_networks        23 non-null     object 
 6   total                 26 non-null     int64  
dtypes: float64(4), int64(2), object(1)
memory usage: 1.5+ KB


Since we want to perform some calculations with the revenue columns later on, I want to make all of them float type. It appears the 'media_networks' column is recognized as an object type. Taking a look at the original dataset, there is a value '4,142' that might make it difficult to convert the 'media_networks' column. Let's replace this value with '4142' and give it a shot. 

In [5]:
# replace '4,142' with '4142' in the 'media_networks' column
revenue = revenue.assign(media_networks = revenue['media_networks'].str.replace('4,142', '4142'))

# cast the media_networks and total columns to float type
revenue = revenue.assign(media_networks = revenue['media_networks'].astype('float'))
revenue = revenue.assign(total = revenue['total'].astype('float'))

revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26 entries, 0 to 25
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  26 non-null     int64  
 1   studio_entertainment  25 non-null     float64
 2   consumer_products     24 non-null     float64
 3   interactive_media     12 non-null     float64
 4   parks_resorts         26 non-null     float64
 5   media_networks        23 non-null     float64
 6   total                 26 non-null     float64
dtypes: float64(6), int64(1)
memory usage: 1.5 KB


There we go.

Finally, we can also see that there are null values in some of the columns. It is worth noting that for some years, there were merges that occurred across certain divisions, which explains some of the null values. 

For the purposes of this study, I will exclude the years 1991 to 1996 as there is no data available for 2 of the divisions (Interactive Media and Media Networks).

To go with a more conservative estimate, I will fill the remaining null values with the value of the prior year.

This also means that the 'total' column must be updated with the new values.

In [6]:
# remove rows 0 to 5, reset the index of the dataframe and drop the new 'index' column
revenue = revenue.loc[6: ].reset_index(drop = True)

# replace null values with the previous year's value
revenue = revenue.fillna(method = 'ffill')

# update the values in the 'total' column to reflect the sum of all revenue columns
revenue = revenue.assign(total = revenue['studio_entertainment'] + revenue['consumer_products'] + revenue['interactive_media']
                        + revenue['parks_resorts'] + revenue['media_networks'])
revenue.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   year                  20 non-null     int64  
 1   studio_entertainment  20 non-null     float64
 2   consumer_products     20 non-null     float64
 3   interactive_media     20 non-null     float64
 4   parks_resorts         20 non-null     float64
 5   media_networks        20 non-null     float64
 6   total                 20 non-null     float64
dtypes: float64(6), int64(1)
memory usage: 1.2 KB


Pefect! Now we can clearly see that the **revenue** table has 20 rows and 7 columns. Each column has 20 non-null values and, apart from the 'year' column which is an integer type, all the revenue columns are of float type.

Let's move onto answering our question of interest.

## Data Analysis

Before we move onto answering the main question of this project, I do want to get an overall sense of the average revenue of each division over the 20 year period of 1997 through 2016. Let's calculate the mean revenue of each of the 5 divisions and create a bar graph to visualize the results.  

In [7]:
# calculate the mean revenue of each division and transform the object into a dataframe
mean_revenue = pd.DataFrame(revenue.loc[:, 'studio_entertainment' : 'media_networks'].mean().reset_index())

# rename the columns 
mean_revenue = mean_revenue.rename(columns = {'index' : 'division', 0 : 'revenue'})

# peek at the new dataframe 
mean_revenue

# use altair to create a bar graph for mean_revenue
mean_revenue_plot = (
    alt.Chart(mean_revenue, width = 450, height = 300)
    .mark_bar(size = 50)
    .encode(
        x = alt.X("division:N", axis = alt.Axis(labels = False), sort = "y", title = "Division"),
        y = alt.Y("revenue:Q", title = "Mean Revenue (millions USD)"),
        color = alt.Color("division", sort = "y", title = "Divisions by Colour")
    )
    .properties(title = "Fig 1. Mean Revenue of Walt Disney's Divisions from 1997 through 2016")
)
mean_revenue_plot

alt.Chart(...)

It appears that media networks has had the greatest mean revenue over the 20 year period of interest. Meanwhile, interactive media seems to have had the smallest mean revenue. However, given that the total revenue has changed from 1997 to 2016, it would still be important to transform the data into proportions before making any comparisons. 

Indeed, I am ultimately interested in understanding the trends in revenue of the 5 divisions as a **proportion** of each year's total revenue. I will thus import and use the script I created with a custom function that updates a list of columns in a dataframe with their proportions out of a specified column representing their totals.

In [8]:
# import the custom script
from prop_calc import prop_calc

# store the division columns in a list 
divisions = ['studio_entertainment', 'consumer_products', 'interactive_media', 'parks_resorts', 'media_networks']

# run prop_calc on the revenue data 
prop_calc(revenue, divisions, 'total')

,year,studio_entertainment,consumer_products,interactive_media,parks_resorts,media_networks,total
0,1997,0.310639,0.168291,0.007743,0.223112,0.290215,22473.0
1,1998,0.298094,0.138971,0.011316,0.240773,0.310846,22976.0
2,1999,0.279805,0.129476,0.008803,0.260918,0.320998,23402.0
3,2000,0.236152,0.102514,0.014498,0.268025,0.378812,25382.0
4,2001,0.274236,0.101410,0.014409,0.235278,0.374667,25540.0
5,2002,0.251586,0.094953,0.014321,0.260381,0.378760,25697.0
6,2003,0.268475,0.085457,0.013416,0.233767,0.398884,27429.0
7,2004,0.279981,0.080688,0.011825,0.249036,0.378470,31120.0
8,2005,0.234804,0.065827,0.011389,0.279246,0.408734,32312.0
9,2006,0.218974,0.063782,0.010703,0.288660,0.417881,34383.0


Now that we have the data in the proper format and the revenue columns for each division as proportions of the total yearly revenue, let's create a line graph for each of the 5 divisions to present how its contribution to Walt Disney's revenue has changed over the years.

In [9]:
# use altair to generate a line plot for studio entertainment
studio_entertainment_plot = (
    alt.Chart(revenue, width = 600, height = 200)
    .mark_line(color = 'teal')
    .encode(
        x = alt.X("year:O", title = "Year"),
        y = alt.Y("studio_entertainment:Q", title = "Proportion of Total Revenue", scale = alt.Scale(domain=[0.00, 0.50])),
    )
    .properties(title = "Fig 2. Revenue of Studio Entertainment")
)

# use altair to generate a line plot for consumer products
consumer_products_plot = (
    alt.Chart(revenue, width = 600, height = 200)
    .mark_line(color = 'gold')
    .encode(
        x = alt.X("year:O", title = "Year"),
        y = alt.Y("consumer_products:Q", title = "Proportion of Total Revenue", scale = alt.Scale(domain=[0.00, 0.50])),
    )
    .properties(title = "Fig 3. Revenue of Consumer Products")
)

# use altair to generate a line plot for interactive media
interactive_media_plot = (
    alt.Chart(revenue, width = 600, height = 200)
    .mark_line(color = 'darkorchid')
    .encode(
        x = alt.X("year:O", title = "Year"),
        y = alt.Y("interactive_media:Q", title = "Proportion of Total Revenue", scale = alt.Scale(domain=[0.00, 0.50])),
    )
    .properties(title = "Fig 4. Revenue of Interactive Media")
)

# use altair to generate a line plot for parks and resorts
parks_resorts_plot = (
    alt.Chart(revenue, width = 600, height = 200)
    .mark_line(color = 'coral')
    .encode(
        x = alt.X("year:O", title = "Year"),
        y = alt.Y("parks_resorts:Q", title = "Proportion of Total Revenue", scale = alt.Scale(domain=[0.00, 0.50])),
    )
    .properties(title = "Fig 5. Revenue of Parks and Resorts")
)

# use altair to generate a line plot for media networks
media_networks_plot = (
    alt.Chart(revenue, width = 600, height = 200)
    .mark_line(color = 'yellowgreen')
    .encode(
        x = alt.X("year:O", title = "Year"),
        y = alt.Y("media_networks:Q", title = "Proportion of Total Revenue", scale = alt.Scale(domain=[0.00, 0.50])),
    )
    .properties(title = "Fig 6. Revenue of Media Networks")
)

# arrange the 5 line plots vertically 
alt.vconcat(studio_entertainment_plot, consumer_products_plot, interactive_media_plot, parks_resorts_plot, media_networks_plot)

alt.VConcatChart(...)

## Discussion

**Summary of Findings**

In this project, I analyzed the Walt Disney revenue dataset and examined the changes in **proportion** of revenue for each division across the 20 year period. 

Before answering this question, I took a look at the mean revenue of each division for the 20 years and found that media networks generated the largest revenue, followed by parks and resorts, studio entertainment, consumer products, and lastly, interactive media. 

While this was interesting, I was ultimately more interested in look at the trends over time. As I expected, the studio entertainment division generated a decreasing proportion of Walt Disney's overall annual revenue. While Disney films have remained iconic across the globe, there is no doubt that the company has experienced significant success in their other business ventures in the past couple of decades. This would have diversified the company's revenue sources and decreased the proportion of revenue studio entertainment alone has contributed. 

Along these lines, the media networks division has generated a significantly increasing proportion of the company's revenue. This is not surprising to me at all given that Disney's media networks division oversees all of its television-related activities, and television has certainly proved to be a formidable challenge to traditional cinema. Similarly, parks and resorts, as well as interactive media, has shown a slight bump in their contributions to Disney's annual revenue. 

What surprised me was the significant drop in consumer product's contribution to Disney's annual revenue over time. I had originally assumed that with globalization, Disney consumer products would have become much more readily available to a greater audience. However, it appears that other factors may have come into play to lead to this decrease. Perhaps, with the rise in competitors, people are faced with an endless selection of alternatives that may be more appealing than the often expensive and exclusive Disney-branded merchandise. 

**Future Directions and Implications**

A question that I would be very interested in exploring is to what extent is Disney's revenue across its divisions affected by external circumstances. For instance, how did the COVID-19 pandemic affect each division's revenue? This would require me to take a look at the data past 2016 and through a few years from now. I would expect that divisions involving physical presence like parks and resorts would have experienced a great drop in revenue proportion, whereas divisions relating to media would have seen a significant increase. Ultimately, understanding trends in revenue as they relate to particular events or changing times is crucial in informing business decisions for any company, let alone one as expansive as Walt Disney. 

# Resources Used
* [Data Source](https://data.world/kgarrett/disney-character-success-00-16)
    * This Disney Revenue database used in this work was curated by **Kelly Garrett**.
* [Data Visualization](https://altair-viz.github.io/user_guide/customization.html#adjusting-axis-labels)
    * Tips and tricks for data visualization was learned from the **Altair Developers**.